# Llama2 모델 기반 vscodeHelper 파인튜닝

드라이브에 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


필요 라이브러리 설치

In [ ]:
### accelerate : hugging face 학습루프 가속화 라이브러리
### peft: LoRA, Prefix Tuning, P-Tuing, Prompt Tuning 과 같은 기법들을 쉽게 사용하도록 나온 라이브러리
### bitsandbytes: gpu 에서 모델을 손쉽게 압축할 수 있는 라이브러리
### trl: TRL (Transformer Reinforcement Learning) 은 transfomer언어 모델의 훈련을 위한 풀스택 라이브러리
!pip install accelerate==0.26.1 peft==0.8.2 bitsandbytes==0.42.0 transformers==4.37.2 trl==0.7.10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 16.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7

패키지 로드

In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments,logging
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
print("load success")

load success


데이터셋 로드

In [ ]:
vscodeHelper_dataset = "codakcodak/vscodeHelper"
dataset = load_dataset(vscodeHelper_dataset, split="train")
print("dataset 확인")
print(dataset[1])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/150 [00:00<?, ? examples/s]

dataset 확인
{'text': '<s>[INST] vscode에서 행 복사를 하려면 어떻게 해야해? [/INST] vscode에서 행 복사는 기본 편집동작이며, ctrl+C으로 수행할 수 있습니다. 명령 ID는 editor.action.clipboardCopyAction입니다. </s>'}


GPU 메모리에 맞는 양자화 설정(4bit 양자화)

In [ ]:
def create_bnb_config():
    compute_dtype = getattr(torch, "float16")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

    return bnb_config

모델 로드 및 토크나이저 로드

In [ ]:
def load_model(model_name, bnb_config):
    n_gpus = torch.cuda.device_count()
    max_memory = f'15GB'

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto", # dispatch efficiently the model on the available ressources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )
    model.config.use_cache = False
    model.config.pretraining_tp = 1

    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

llama2기반의 한국어 사전 학습 모델 및 토크나이저 로드

In [ ]:
model_name = "beomi/llama-2-ko-7b"
bnb_config = create_bnb_config()
model, tokenizer = load_model(model_name, bnb_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00015.safetensors:   0%|          | 0.00/919M [00:00<?, ?B/s]

model-00002-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

peft의 lora 값 설정

In [ ]:
def create_peft_config():
    """
    Create Parameter-Efficient Fine-Tuning config for your model
    :param modules: Names of the modules to apply Lora to
    """
    peft_params = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
    )

    return peft_params

학습 옵션 설정

In [ ]:
def create_training_config():
    training_params = TrainingArguments(
        output_dir="./results",
        num_train_epochs=10,
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        save_steps=25,
        logging_steps=25,
        learning_rate=2e-4,
        weight_decay=0.001,
        fp16=False,
        bf16=False,
        max_grad_norm=0.3,
        max_steps=-1,
        warmup_ratio=0.03,
        group_by_length=True,
        lr_scheduler_type="constant",
        report_to="tensorboard"
    )
    return training_params

SFTTrainer 학습 객체 생성

In [ ]:
peft_params=create_peft_config()
training_params=create_training_config()
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/150 [00:00<?, ? examples/s]

학습

In [ ]:
trainer.train()

Step,Training Loss
25,2.205300
50,0.771100
75,0.499200
100,0.365500
125,0.269800
150,0.223500
175,0.179700


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=190, training_loss=0.6074640136016043, metrics={'train_runtime': 896.9839, 'train_samples_per_second': 1.672, 'train_steps_per_second': 0.212, 'total_flos': 3998546591612928.0, 'train_loss': 0.6074640136016043, 'epoch': 10.0})

학습결과 드라이브에 저장

In [ ]:
import shutil
shutil.copytree("/content/results","/content/drive/MyDrive/vscodeHelperResult")

'/content/drive/MyDrive/vscodeHelperResult'

모델 언로드

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

최적의 checkpoint 모델 찾기(gpu의 캐시를 비우면서 반복)

In [ ]:
def load_model_without_tokenizer(model_name, bnb_config):
    n_gpus = torch.cuda.device_count()
    max_memory = f'15GB'

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto", # dispatch efficiently the model on the available ressources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )
    model.config.use_cache = False
    model.config.pretraining_tp = 1

    return model

model_name = "beomi/llama-2-ko-7b"
bnb_config = create_bnb_config()
model = load_model_without_tokenizer(model_name, bnb_config)

#체크포인트 번호를 바꿔가며 가장 좋은 모델 선택
adaptor="/content/drive/MyDrive/vscodeHelperResult/checkpoint-175"
model.load_adapter(adaptor)

tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

프롬프트 질문 테스트

In [ ]:
logging.set_verbosity(logging.CRITICAL)
prompt = "vscode에서 아래에 행을 삽입할려면 어떻게 해야해?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] vscode에서 아래에 행을 삽입할려면 어떻게 해야해? [/INST] vscode에서 아래에 행을 삽입은 기본 편집동작이며, ctrl+shift+Enter으로 수행할 수 있습니다. 명령 ID는 editor.action.insertLineAfter입니다. 2021/02/19 16:52 [INST] vscode에서 아래에 행을 삽입할려면 어떻게 해야해? vscode


로컬에 최적의 모델 어댑터 및 토크나이저 업로드

In [ ]:
import os
model_name="llama2-7b-ko-4bit-vscodeHelper-adapters"
os.makedirs(model_name, exist_ok = True)
model.save_pretrained(model_name)
tokenizer.save_pretrained(model_name)

import shutil
shutil.copytree("/content/"+model_name,"/content/drive/MyDrive/"+model_name)

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


'/content/drive/MyDrive/llama2-7b-ko-4bit-vscodeHelper-adapters'

허깅페이스에 모델 업로드

In [ ]:
from huggingface_hub import interpreter_login
interpreter_login()

model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/codakcodak/llama2-7b-ko-4bit-vscodeHelper-adapters/commit/f06af96b8ee3938006bf3387eebf329eafc27516', commit_message='Upload tokenizer', commit_description='', oid='f06af96b8ee3938006bf3387eebf329eafc27516', pr_url=None, pr_revision=None, pr_num=None)